# Analyze Index Stock Performance
SPX, QQQ, DOW

In [1]:
!pip install requests pandas yfinance investpy lxml beautifulsoup4 polygon-api-client investpy tabulate openpyxl

!pip install gspread gspread-dataframe oauth2client

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor


## Fetch and save index components

In [3]:
# Fetch index components 
def fetch_components_slickcharts(index):
    base_url = "https://slickcharts.com/"
    index_map = {
        'QQQ': 'nasdaq100',
        'SPX': 'sp500',
        'DOW': 'dowjones',
        'IWM': 'russell2000'
    }
    if index.upper() not in index_map:
        raise ValueError("Index must be either 'QQQ', 'SPX', 'DOW', or 'IWM'")
    
    url = f"{base_url}{index_map[index.upper()]}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find('table', {'class': 'table'})
    
    data = []
    headers = []

    for i, row in enumerate(table.find_all('tr')):
        cols = [ele.text.strip() for ele in row.find_all('td')]
        if i == 0:
            headers = [ele.text.strip() for ele in row.find_all('th')]
        else:
            if len(cols) > 1:  # Ensure there is data in the row
                data.append(cols)
    
    df = pd.DataFrame(data, columns=headers)
    return df

In [4]:
# Append components with industry info
from collections import Counter
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

def fetch_industry_info(symbol):
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        industry = info.get("industry", "N/A")
        #print(f"Fetched industry for {symbol}: {industry}")
        return industry
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return "N/A"

def append_industry_info(df):
    industries = []
    for _, row in df.iterrows():
        symbol = row['Symbol']
        industry = fetch_industry_info(symbol)
        industries.append(industry)
    
    df['Industry'] = industries
    return df

def print_all_company_details_compact(df, rank_by='weight'):
    # Print DataFrame columns for debugging
    print("DataFrame columns:", df.columns)
    
    # Convert 'Weight' or 'Portfolio %' column to numeric if it exists
    weight_column = 'Weight' if 'Weight' in df.columns else 'Portfolio%' if 'Portfolio%' in df.columns else None
    
    if rank_by == 'weight' and weight_column:
        df[weight_column] = df[weight_column].astype(str).str.replace('%', '').astype(float)
        df = df.sort_values(by=weight_column, ascending=False)
    elif rank_by == 'industry' and 'Industry' in df.columns:
        df = df.sort_values(by='Industry')
    
    # Get the maximum length of each column for formatting
    col_widths = [max(df[col].astype(str).map(len).max(), len(col)) for col in df.columns]
    format_str = ' | '.join([f'{{:<{width}}}' for width in col_widths])
    
    print(format_str.format(*df.columns))
    print('-' * (sum(col_widths) + 3 * (len(col_widths) - 1)))
    
    for index, row in df.iterrows():
        print(format_str.format(*row))

from tabulate import tabulate

def display_industry_stats(df):
    industry_stats = df['Industry'].value_counts().reset_index()
    industry_stats.columns = ['Industry', 'Number of Companies']
    industry_stats = industry_stats.sort_values(by='Industry')
    
    data = []
    for industry in industry_stats['Industry']:
        tickers = df[df['Industry'] == industry]['Symbol'].tolist()
        ticker_string = ", ".join(tickers)
        data.append([industry, len(tickers), ticker_string])
    
    print(tabulate(data, headers=["Industry", "Number of Companies", "Tickers"], tablefmt="grid", stralign="left"))


In [37]:
df_qqq=append_industry_info(fetch_components_slickcharts('qqq'))
df_spx=append_industry_info(fetch_components_slickcharts('spx'))
# df_qqq=append_industry_info(fetch_components_slickcharts('qqq'))

In [89]:
def save_dataframe_to_csv(df, filename):
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

def save_dataframe_to_excel(df, filename):
    df.to_excel(filename, index=False)
    print(f"Data saved to {filename}")

# Save the dataframes to files
save_dataframe_to_excel(df_qqq, 'df_qqq.xlsx')
save_dataframe_to_excel(df_spx, 'df_spx.xlsx')


Data saved to df_qqq.xlsx
Data saved to df_spx.xlsx


## Understand Index Components

### QQQ Components

In [53]:
#df_qqq = load_dataframe_from_file('df_qqq.pkl')

# Print details of all companies in a compact format
print_all_company_details_compact(df_qqq, rank_by='industry')
display_industry_stats(df)

DataFrame columns: Index(['#', 'Company', 'Symbol', 'Portfolio%', 'Price', 'Chg', '% Chg', '',
       'Industry'],
      dtype='object')
#   | Company                             | Symbol | Portfolio% | Price    | Chg    | % Chg    |  | Industry                           
---------------------------------------------------------------------------------------------------------------------------------------
60  | Ross Stores Inc                     | ROST   | 0.33       | 143.99   | -0.41  | (-0.28%) |  | Apparel Retail                     
78  | Lululemon Athletica Inc             | LULU   | 0.24       | 300.18   | -1.49  | (-0.49%) |  | Apparel Retail                     
9   | Tesla Inc                           | TSLA   | 2.62       | 247.71   | 16.45  | (7.11%)  |  | Auto Manufacturers                 
86  | Coca-Cola Europacific Partners PLC  | CCEP   | 0.22       | 73.58    | 0.84   | (1.15%)  |  | Beverages - Non-Alcoholic          
57  | Monster Beverage Corp               | MNS

### SPX Components

In [55]:
#df_spx = load_dataframe_from_file('df_spx.pkl')

# Print details of all companies in a compact format
print_all_company_details_compact(df_spx, rank_by='industry')
display_industry_stats(df)

DataFrame columns: Index(['#', 'Company', 'Symbol', 'Portfolio%', 'Price', 'Chg', '% Chg',
       'Industry'],
      dtype='object')
#   | Company                                        | Symbol | Portfolio% | Price    | Chg    | % Chg    | Industry                                
----------------------------------------------------------------------------------------------------------------------------------------------------
452 | The Interpublic Group of Companies, Inc.       | IPG    | 0.02       | 28.77    | 0.20   | (0.70%)  | Advertising Agencies                    
363 | Omnicom Group Inc.                             | OMC    | 0.04       | 89.19    | 0.11   | (0.12%)  | Advertising Agencies                    
465 | Huntington Ingalls Industries, Inc.            | HII    | 0.02       | 246.19   | -1.29  | (-0.52%) | Aerospace & Defense                     
382 | Textron, Inc.                                  | TXT    | 0.04       | 86.17    | 0.39   | (0.45%)  | Aerospace & De

## Calculate performance 

In [76]:
def calculate_performance(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    
    if len(hist) == 0:
        return None
    
    start_price = hist['Close'].iloc[0]
    end_price = hist['Close'].iloc[-1]
    performance = (end_price - start_price) / start_price
    return performance

def get_performance_data(symbol, company, industry, start_date, end_date):
    try:
        performance = calculate_performance(symbol, start_date, end_date)
        if performance is not None:
            return (symbol, company, performance, industry)
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
    return (symbol, company, None, industry)  # Ensure it returns a tuple with four elements

def fetch_all_performance_data(df, start_date, end_date):
    performance_data = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(get_performance_data, row['Symbol'], row['Company'], row['Industry'], start_date, end_date) for index, row in df.iterrows()]
        for future in as_completed(futures):
            result = future.result()
            if result and result[2] is not None:  # Check if performance is not None
                performance_data.append(result)
    
    performance_data.sort(key=lambda x: x[2], reverse=True)
    return performance_data

def display_top_bottom_stocks(df, period, top_n, bottom_n, start_date=None, end_date=None):
    if not end_date:
        end_date = datetime.now()
    
    if start_date and end_date:
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
    else:
        if period == 'ytd':
            start_date = datetime(end_date.year, 1, 1)
        else:
            delta = {
                '1d': 1,
                '5d': 5,
                '1m': 30,
                '6m': 182,
                '1y': 365,
                '2y': 365*2,
                '3y': 365*3,
                '5y': 365*5,
                '10y': 365*10,
                '20y': 365*20
            }[period]
            start_date = end_date - timedelta(days=delta)
    print("Start and end dates:", start_date, end_date)
    
    performance_data = fetch_all_performance_data(df, start_date, end_date)
    
    print(f"Top {top_n} Stocks:")
    top_industries = []
    for symbol, company, performance, industry in performance_data[:top_n]:
        print(f"{symbol} ({company} - {industry}): {performance*100:.1f}%")
        top_industries.append(industry)
    
    print(f"\nBottom {bottom_n} Stocks:")
    bottom_industries = []
    for symbol, company, performance, industry in performance_data[-bottom_n:]:
        print(f"{symbol} ({company} - {industry}): {performance*100:.1f}%")
        bottom_industries.append(industry)
    
    top_industry_count = Counter(top_industries)
    bottom_industry_count = Counter(bottom_industries)
    
    print(f"\nTop {top_n} Stocks by Industry:")
    for industry, count in top_industry_count.items():
        print(f"{industry}: {count} companies")
    
    print(f"\nBottom {bottom_n} Stocks by Industry:")
    for industry, count in bottom_industry_count.items():
        print(f"{industry}: {count} companies")

### QQQ Performance

In [86]:
# Example usage:
# Parameters
index = 'QQQ'  # Change to 'QQQ' for Nasdaq-100, 'DOW' for Dow Jones, 'IWM' for Russell 2000
df = df_qqq
period = '1y'  # Options: 'ytd', '1d', '5d', '1m', '6m', '1y', '2y', '3y', '5y', '10y', '20y'
top_n = 20
bottom_n = 20
start_date = '1999-01-01'  # Example: '2004-01-01'
end_date = '1999-12-31'  # Example: '2024-12-31'


# Display top and bottom performing stocks
display_top_bottom_stocks(df, period, top_n, bottom_n, start_date, end_date)

Start and end dates: 1999-01-01 00:00:00 1999-12-31 00:00:00


TSLA: Data doesn't exist for startDate = 915166800, endDate = 946616400
META: Data doesn't exist for startDate = 915166800, endDate = 946616400
GOOG: Data doesn't exist for startDate = 915166800, endDate = 946616400
NFLX: Data doesn't exist for startDate = 915166800, endDate = 946616400
GOOGL: Data doesn't exist for startDate = 915166800, endDate = 946616400
AVGO: Data doesn't exist for startDate = 915166800, endDate = 946616400
TMUS: Data doesn't exist for startDate = 915166800, endDate = 946616400
ISRG: Data doesn't exist for startDate = 915166800, endDate = 946616400
PDD: Data doesn't exist for startDate = 915166800, endDate = 946616400
PANW: Data doesn't exist for startDate = 915166800, endDate = 946616400
CRWD: Data doesn't exist for startDate = 915166800, endDate = 946616400
MDLZ: Data doesn't exist for startDate = 915166800, endDate = 946616400
MELI: Data doesn't exist for startDate = 915166800, endDate = 946616400
NXPI: Data doesn't exist for startDate = 915166800, endDate = 94

Top 20 Stocks:
QCOM (QUALCOMM Inc - Semiconductors): 2234.2%
LRCX (Lam Research Corp - Semiconductor Equipment & Materials): 441.4%
BIIB (Biogen Inc - Drug Manufacturers - General): 302.0%
CTSH (Cognizant Technology Solutions Corp - Information Technology Services): 277.1%
ASML (ASML Holding NV - Semiconductor Equipment & Materials): 233.6%
ADI (Analog Devices Inc - Semiconductors): 200.2%
AMAT (Applied Materials Inc - Semiconductor Equipment & Materials): 180.7%
CPRT (Copart Inc - Specialty Business Services): 179.4%
ADBE (Adobe Inc - Software - Infrastructure): 164.9%
INTU (Intuit Inc - Software - Application): 144.6%
AAPL (Apple Inc - Consumer Electronics): 143.2%
AMGN (Amgen Inc - Drug Manufacturers - General): 141.7%
CSGP (CoStar Group Inc - Real Estate Services): 140.6%
KLAC (KLA Corp - Semiconductor Equipment & Materials): 137.6%
NVDA (NVIDIA Corp - Semiconductors): 135.6%
TXN (Texas Instruments Inc - Semiconductors): 128.8%
CSCO (Cisco Systems Inc - Communication Equipment): 12

### SPX Performance

In [83]:
# Example usage:
# Parameters
index = 'SPX'  # Change to 'QQQ' for Nasdaq-100, 'DOW' for Dow Jones, 'IWM' for Russell 2000
df = df_spx
period = '2y'  # Options: 'ytd', '1d', '5d', '1m', '6m', '1y', '2y', '3y', '5y', '10y', '20y'
top_n = 30
bottom_n = 30
start_date = '2019-01-01'  # Example: '2004-01-01'
end_date = '2019-12-31'  # Example: '2024-12-31'

# Display top and bottom performing stocks
display_top_bottom_stocks(df, period, top_n, bottom_n, start_date, end_date)

Start and end dates: 2019-01-01 00:00:00 2019-12-31 00:00:00


$BRK.B: possibly delisted; No timezone found
ABNB: Data doesn't exist for startDate = 1546318800, endDate = 1577768400
CEG: Data doesn't exist for startDate = 1546318800, endDate = 1577768400
CARR: Data doesn't exist for startDate = 1546318800, endDate = 1577768400
GEV: Data doesn't exist for startDate = 1546318800, endDate = 1577768400
OTIS: Data doesn't exist for startDate = 1546318800, endDate = 1577768400
KVUE: Data doesn't exist for startDate = 1546318800, endDate = 1577768400
GEHC: Data doesn't exist for startDate = 1546318800, endDate = 1577768400
$MTD: possibly delisted; No price data found  (1d 2019-01-01 00:00:00 -> 2019-12-31 00:00:00)


$MTD: possibly delisted; No price data found  (1d 2019-01-01 00:00:00 -> 2019-12-31 00:00:00)


VLTO: Data doesn't exist for startDate = 1546318800, endDate = 1577768400
$BF.B: possibly delisted; No price data found  (1d 2019-01-01 00:00:00 -> 2019-12-31 00:00:00)


$BF.B: possibly delisted; No price data found  (1d 2019-01-01 00:00:00 -> 2019-12-31 00:00:00)


SOLV: Data doesn't exist for startDate = 1546318800, endDate = 1577768400


Top 30 Stocks:
ENPH (Enphase Energy, Inc. - Solar): 415.7%
AMD (Advanced Micro Devices - Semiconductors): 141.7%
PODD (Insulet Corporation - Medical Devices): 130.6%
BLDR (Builders Firstsource, Inc. - Building Products & Equipment): 120.9%
PAYC (Paycom Software, Inc. - Software - Application): 119.2%
TER (Teradyne, Inc. - Semiconductor Equipment & Materials): 118.3%
LRCX (Lam Research Corp - Semiconductor Equipment & Materials): 116.1%
FICO (Fair Isaac Corporation - Software - Application): 101.9%
DAY (Dayforce, Inc. - Software - Application): 101.5%
KLAC (Kla Corporation - Semiconductor Equipment & Materials): 101.0%
TGT (Target Corporation - Discount Stores): 100.2%
GNRC (Generac Holdings Inc - Specialty Industrial Machinery): 98.3%
MTCH (Match Group, Inc - Internet Content & Information): 95.8%
BX (Blackstone Inc. - Asset Management): 94.8%
LDOS (Leidos Holdings, Inc. - Information Technology Services): 91.6%
CPRT (Copart Inc - Specialty Business Services): 90.6%
QRVO (Qorvo, Inc. -